# Introduction

In this document, we 

1. Recap the theory behind variational auto-encoders
1. Recap the practice of convolutional neural networks
1. Describe how to implement a VAE with particular emphasis on
  1. The additional of regularization terms (and fragility of the graph)
  1. The appropriate way to scale the KL divergence
  1. The necessity of monitoring and check-pointing during training
  1. The use of `GradientTape` as a debugging tool


# Theory of a Variational Auto-Encoder

Auto-encoding is a form of dimensionality reduction. The intent is either to obtain a more parsimonious representation of the data for subsequent analysis or -- in the generative case -- to learn a low-dimensional representation from which samples can easily be obtained, in order to -- via a reverse-transformation -- obtain samples in the high-dimensional space. In the case of generative networks, one can endeavour to assign meanings to dimensions post-hoc, and vary them to transform images.


## Bayesian Linear Case: PCA

In the Bayesian case, we focus on the generative story: which means one starts with sampling:

$$
\begin{align}
z_n & \sim N_P(0, I) & x_n \sim N_F(W z_n + \mu, \sigma^2 I)
\end{align}
$$
where $W \in \mathbb{R}^{F \times P}$. If the data are centred to have zero-mean then we can remove $\mu$. In a fully Bayesian framework, this boils down pPCA where the inference procedure utilises EM. 

In a frequentist frame-work, you're looking at PCA, where you're trying to find an orthonormal basis $\boldsymbol{w}_1, \ldots, \boldsymbol{w}_p$ , so $\boldsymbol{w}_i^\top\boldsymbol{w}_j = 0$ for all $i \neq j$ and $\boldsymbol{w}_i^\top \boldsymbol{w}_i = 1$. Inference proceeds using the reconstruction error

$$
\begin{align}
\xi(\boldsymbol{Z}, \boldsymbol{W}) &  = \frac{1}{N} (X - ZW^\top)^\top(X - ZW^\top) \\
& = \sum_P \frac{1}{N} (X - \boldsymbol{z}_p \boldsymbol{w_p}^\top)^\top(X - \boldsymbol{z}_p \boldsymbol{w_p}^\top)\\
& = \sum_P \frac{1}{N} \sum_n (\boldsymbol{x}_n - z_{np} \boldsymbol{w}_p)^2
\end{align}
$$
Setting $p=1$ to consider the first of the $P$ components, we end up with the specific error

$$
\begin{align}
& &\xi(z_{n1}, \boldsymbol{w}_1) & = \frac{1}{N} \sum_n (\boldsymbol{x}_n - z_{n1} \boldsymbol{w}_1)^2 \\
& \implies &  \frac{d\xi(z_{n1}, \boldsymbol{w}_1)}{d z_{n1}} & = -\frac{2}{N} (w_1^\top x_n - z_{n1}) = 0 \\
& \implies & z_{n1} & = w_1^\top x_n
\end{align}
$$
Note that the quadratic term in the error $z_i^\top w_i^\top w_i z_i = z_i z_i$ due to orthonormality of $w_i$.

If you replace $z_{1n}$ with $w_1^\top x_n$ and work it out eventually you get

$$
\begin{align}
\xi(w_1) & = \frac{1}{N} \sum_n x_n^\top x_n - w_1^\top x_n x_n^\top w_q \\
& \propto - \frac{1}{N} w_1^\top X^\top X w_1
\end{align}
$$

Since $w_1$ is orthornomal and given the assumption that X are zero-mean, this means we have to minimise the negative covariance, or put another way, _maximise_ the covariance.

An interesting aside: our maximisation objective $\frac{1}{N}\sum_n w_1^\top x_n x_n^\top w_1 = \frac{1}{N}\sum_n z_{1n}^2$

Using Lagrange terms to add in the orthogonality constraint we end up with

$$
\begin{align}
& \frac{d}{d w_1} w_1^\top \hat{S} w_1 - \lambda_1 w_1^\top w_1 \\
& = \hat{S} w_1 - \lambda_1 w_1
\end{align}
$$
where $\hat{S} = X^\top X$ is the sample covariance and this becomes an eigen-value problem. Working through fully (see Girolami's notes) you end up with the procedure of creating an eigen-decomposition of $\hat{S} = WDW$ in which $Z = XW$

**FIXME: The rest of that lecture, plus the additions with respect to linear regression and CCA, are well worth a read.**

## Simple Machine Learning Case

Define two functions $f: \mathbb{R}^{F} \rightarrow \mathbb{R}^{P}$ and $g: \mathbb{R}^{P} \rightarrow \mathbb{R}^{F}$ such that $g \approx f^{-1}$. Hence our transform is $f(x; w) = z$ and our reverse transform is $g(z; u) = x$.

To find a good solution it suffices to define an pseudo-identity function $\lambda(x; w, u) = g(f(x_n; w); u)$, or more concisely $\lambda = f \odot g$, and then an appropriate error function which can be minimised to obtain parameter estimates:

$$
\xi(u, w) = \frac{1}{N}\sum_n \left(x_n - \lambda(x_n; w, u)\right)^2
$$

In this specific case, $f(x; w)$ may be a convolutional network using a succession of batch-normalisation, convolution, and dropout layer triples; and $g(x; u)$ may be a convolutional network using a succession of batch-normalisation, convolution-transpose and dropout layer triples. 

Absent any regularization, there is no way of predicting what values $z$ may take. One could add a regularization term,

$$
\xi(u, w; \sigma^{-2}) = \frac{1}{N}\sum_n \left(x_n - \lambda(x_n; w, u)\right)^2 + \sigma^{-2} \cdot f(x_n; w)^\top f(x_n; w)
$$

but while this controls the _location_ of $z$, it does not control the _spread_. 

Note that in a Bayesian case $\sigma^2$ is the spread around the end result, i.e. $x \sim N((f \odot g)(x), \sigma^2)$. It is probably interesting to pursue the Bayesian idea of this parametric model a little further before proceeding to the fully variational case.

$$
\begin{align}
z_n|x_n & \sim N(f(x_n; w), I) &
x_n|z_n & \sim N(g(z_n; u), \sigma^2)
\end{align}
$$
Of course, due to its recursive nature, this is not a proper model.

We could go one step further and let the variance float in which case

$$
\begin{align}
\mu_n|x_n & \sim N(f(x_n; w), I) &
\ln \tau^2_n|x_n & \sim N(h(x_n; v), I) \\
z_n|\mu_n, \tau^2_n & \sim N(\mu_n, \tau^2_n) &
x_n|z_n & \sim N(z_n, \sigma^2)
\end{align}
$$

where via the prior we've encoded a preference for the variance to be close to zero. While this is _still_ not a valid model (due to it's recursive nature) it _does_ solve the problem of knowing in advance of model fitting where the model spread and location are going to be. In the case where we're using convolutional nets, we may use a fairly primitive form of transfer learning and instead write

$$
\binom{\mu_n}{\ln tau^2_n} \sim N(f(x_n; w), I)
$$

Yet none of this is a problem model. How would one generate a proper model? The key is _parametric posterior inference_



## Variational Case

This was outlined in the paper [Auto-Encoding Variational Bayes](https://arxiv.org/abs/1312.6114v10)

The variational model is as follows:

$$
\begin{align}
z_n & \sim p(z; \theta) &
x_n|z_n & \sim p(x|z; \theta)
\end{align}
$$
This is a very loose definition. The only constraint is that the two distributions over $z$ and $x$ come from parameteric families, jointly specified using $\theta$; and additionally that the PDFs are differentiable a.e. with respect to $\theta$ and $z$ i.e. we can derive a workable variational inference algorithm.

One thing that should leap out here, is that the parameters of the _posterior_ distribution over $z_n$ will be a function of $\theta$ and $x_n$, thereby creating a link that we will want to employ: i.e. $f = p(z|x; \phi(x, \theta))$ and g = $p(x|z; \theta)$. The use of _parametric_ distributions allows us to go back and forth like this.

In some cases it may not be possible to trivially use Bayes theorem to obtain the posterior, the _evidence_ $p(x, \psi(\theta)$ may not be available in a tractable form and will need to be approximated -- in this case in a deterministic fashion. The evidence _is_ useful in applications, it can be used in de-noising apparently... **FIXME**

The other thing to be worried about is how to control the spread over the posterior distribution of $z$. One method is just explicity define priors over all parameters, and let the mechanics of posterior variational inference handle the rest. The approach in this paper however, is simply to directly add a regularization in the form of a KL divergence between the posterior and the ideal distribution, as a _structured approximation_ in order to avoid making factorization assumptions. The inference procedure in particular will allow for joint inference of $\phi$ and $\theta$

**FIXME: Why does it do this....**

### Inference

#### Step 1/4: Defining the Objective Function

So we have $p(z|\theta)$ and $p(x|z; \theta)$, where the expection of the latter is equivalent to $g(z)$. The posterior $p(z|x; \phi(x, \theta))$, if it were available, would be equivalent to $f(x)$, but since it's not we use an approximation $q(z|x; \phi)$ where $\phi$ is the value of the posterior parameters.

Sticking with the structured approach, we go directly to the equations for the variational bound.

$$
\begin{align}
\ln p(x) & = \ln \int_z p(x, z; \theta) \\
& = \ln \int_z p(x, z; \theta) \frac{q(z|x; \phi)}{q(z|x; \phi)} \\
& = \ln \int_z \frac{p(x, z; \theta)}{q(z|x; \phi)} {q(z|x; \phi)} \\
& = \ln \mathbb{E}_{q(z|x; \phi)} \left[\frac{p(x, z; \theta)}{q(z|x; \phi)} \right] \\
& \geq \mathbb{E}_{q(z|x; \phi)} \left[\ln \frac{p(x, z; \theta)}{q(z|x; \phi)} \right] \\
& = \mathbb{E}_{q(x|z;\phi)}\left[
    \ln p(x|z; \theta) + \ln p(z; \theta)
  \right] + H[q]\\
& = \mathcal{L}(\theta, \phi; x)
\end{align}
$$


where from Jensen we get $\mathbb{E}[f(x)] \leq f(\mathbb{E}[x])$ for convex functions $f$, such as $\ln(\cdot)$. 

Re-arranging the terms, one can separately obtain the solution

$$
\begin{align}
\ln p(x) & \geq \mathcal{L}(\theta, \phi; x) \\
  & = \mathbb{E}_{q(x|z;\phi)}\left[
    \ln p(x|z; \theta)\right] 
    + \mathbb{E}_{q(x|z;\phi)}\left[\ln p(z; \theta)\right] 
    - \mathbb{E}_{q(x|z;\phi)}\left[\ln q(x|z;\phi) \right] \\
& = \mathbb{E}_{q(x|z;\phi)}\left[
    \ln p(x|z; \theta)\right] 
    - D_{KL}[q(x|z;\phi) || p(z;\theta)]
\end{align}
$$

where $D_{KL}[q||p] = \int_z q(z)\ln \frac{q(z)}{p(z)} = \mathbb{E}_q[\ln q(z)] - \mathbb{E}_q[\ln p(z)]$

Observing that by definition $\mathbb{E}_{q(x|z;\phi)}\left[\ln p(x|z; \theta)\right] = \ln p(x; \theta)$ this means that 

$$
\begin{align}
& \mathcal{L}(\theta, \phi; x) &=&  \ln p(x; \theta) - D_{KL}[q(x|z;\phi) || p(z;\theta)] \\
\implies & \ln p(x; \theta) &=& \mathcal{L}(x; \theta, \phi) + D_{KL}[q(x|z;\phi) || p(z;\theta)]
\end{align}
$$

Ultimately our objective function is $\mathcal{L}(\theta, \phi; x)$.

It's worth emphasising at this point that we will be using a deep convolutional network to learn the parameters $\phi$ of $q(z|x; \phi)$, and that we will additinonally be using a deep convolutional-transpose net to learn the paremters $\theta$ of $p(x|z; \theta)$

#### Step 2/4: Obtaining Derivatives and Solving

A complicating factor in the presentation is that the authors don't want to us _analytical_ definitions of $p(x|z; \theta)$, $p(z; \theta)$ or $q(z|x;\phi)$ yet.

The authors suggest that taking the derivatives with respect to $\theta$ and solving is straightforward, as $\nabla_{\theta} \mathbb{E}_{q(z; \phi)}\left[f(\theta, \phi)\right] = \mathbb{E}_{q(z; \phi)}\left[\nabla_{\theta} f(\theta, \phi)\right]$ which in the worst-case scenario can be evaluated using sampling.

This is not necessarily true with $\phi$, but there are ways to ensure the derivative of the expectation is itself an expectation:

$$
\begin{align}
\nabla_\phi \mathbb{E}_{q(z;\phi)}[f(z)]
 &= \nabla_\phi \int_z f(z) q(z; \phi) \\
 &= \int_z f(z) \nabla_\phi q(z; \phi) \\
 &= \int_z f(z) \frac{q(z; \phi)}{q(z; \phi)} \nabla_\phi q(z; \phi) \\
 & = \int_z f(z) q(z; \phi) \nabla_\phi \ln q(z; \phi) \\
 & = \mathbb{E}_{q(z;\phi)}[f(z)\nabla_\phi \ln q(z; \phi)] \\
 & \approx \frac{1}{S}\sum_s f(z^{(s)})\nabla_\phi \ln q(z^{(s)}; \phi) & z^{(s)} \sim q(z; \phi)
\end{align}
$$
which exploits the fact that since $\frac{d}{dx} \ln f(x) = \frac{1}{f(x)}\frac{d}{dx}f(x)$ then $\frac{1}{f(x)}\frac{d}{dx}f(x)$ = $\frac{d}{dx}\ln f(x)$. This apparently has high variance, though the [**FIXME**  citation supporting that claim](FIXME) doesn't explain why.

A second method used to obtain the same outcome -- ensure the derivative of the expection is an expection -- is to employ an auxilliary variable.

The core idea is a second-order approximation: g(x



# Appendix

## Definitions

Convolutional Layer


Convolutional Transpose Layer


Batch normalisation, what is it, what does it do

# How to Reduce Sampling Variance

Use the real distribution with enough samples
$$
\begin{align}
\mathbb{E}[f(x)] & \approx \frac{1}{S}\sum_s f(x^{(s)})
& x^{(s)} \sim p(x)
\end{align}
$$

Use an approximate distribution that's sufficiently close with enough samples.
$$
\begin{align}
\mathbb{E}[f(x)] & \approx \frac{1}{S}\sum_s f(x^{(s)})
& x^{(s)} \sim q(x)
\end{align}
$$
Only works if $q(x)$ is wider than $p(x)$ and non-zero wherever $p(x)$ is non-zero.

Use an approximate distribution, but re-weight its samples' importance to the final solution to reflect where the real and approximate distributions differ

$$
\begin{align}
\mathbb{E}[f(x)] & \approx \frac{1}{S}\sum_s f(x^{(s)})\frac{p(x)}{q(x)}
& x^{(s)} \sim p(x)
\end{align}
Only works if $q(x)$ is wider than $p(x)$ and non-zero wherever $p(x)$ is non-zero.


